<a href="https://colab.research.google.com/github/Tecpandas/comicgenerator_AI/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install -q streamlit diffusers transformers accelerate pillow pyngrok

In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())


GPU Available: True


In [40]:
!pip install gTTS

In [48]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
import torch
from gtts import gTTS
from io import BytesIO

# Load models with caching
@st.cache_resource
def load_caption_generator():
    return pipeline("text2text-generation", model="google/flan-t5-small")

@st.cache_resource
def load_image_generator():
    return StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16
    ).to("cuda")

caption_gen = load_caption_generator()
image_pipe = load_image_generator()

# Utilities
def generate_caption(scene):
    output = caption_gen(f"Generate a comic panel caption: {scene}", max_length=30)
    return output[0]['generated_text']

def generate_image(prompt, style):
    style_map = {
        "Cartoon": "in a cartoon style",
        "Comic": "in a Marvel comic style",
        "Anime": "in an anime style",
        "Default": ""
    }
    full_prompt = f"{prompt}, {style_map.get(style, '')}"
    result = image_pipe(prompt=full_prompt, guidance_scale=8.0, height=768, width=768)
    return result.images[0]

def add_caption_to_image(img, text):
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 28)
    except:
        font = ImageFont.load_default()

    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    margin = 10
    draw.rectangle([(0, 0), (img.width, text_height + 2 * margin)], fill=(0, 0, 0, 180))
    draw.text((margin, margin), text, font=font, fill="white")
    return img

def combine_panels_to_page(panels, panels_per_page=4):
    pages = []
    for i in range(0, len(panels), panels_per_page):
        chunk = panels[i:i + panels_per_page]
        width = sum(p.width for p in chunk)
        height = max(p.height for p in chunk)
        page = Image.new("RGB", (width, height))
        x_offset = 0
        for p in chunk:
            page.paste(p, (x_offset, 0))
            x_offset += p.width
        pages.append(page)
    return pages

def generate_voice(caption):
    try:
        tts = gTTS(text=caption)
        mp3_fp = BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        return mp3_fp.read()
    except Exception:
        return None

# Dark Theme Styling
st.set_page_config(page_title="AI Comic Generator", layout="centered")

st.markdown("""
    <style>
        body {
            background-color: #121212;
            color: white;
        }
        .stTextInput > div > div > input,
        .stTextArea textarea,
        .stSelectbox div[data-baseweb="select"] {
            background-color: #2b2b2b;
            color: white;
        }
        .stButton > button {
            background-color: #ff4b4b;
            color: white;
            border: none;
            font-weight: bold;
        }
        .stButton > button:hover {
            background-color: #ff6666;
        }
    </style>
""", unsafe_allow_html=True)

st.markdown("<h1 style='text-align: center; color: #FF4B4B;'>🎨 AI Comic Generator</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; color: #cccccc;'>Turn your imagination into a comic strip with AI!</p>", unsafe_allow_html=True)

with st.expander("📖 Example Prompt"):
    st.code("""
A teenager is bitten by a radioactive spider in a school science lab.
He wakes up with strange powers — sticking to walls and super strength.
At first, he uses his powers to become famous and earn money.
One night, he lets a criminal escape out of selfishness.
Later, he discovers that same criminal has killed his Uncle Ben.
The teen vows to fight crime, remembering his uncle’s words: "With great power comes great responsibility."
""")

# Inputs
story = st.text_area("✍️ Enter your story (one scene per sentence or line):", height=200)
style_select = st.selectbox("🎨 Choose a visual style:", ["Default", "Cartoon", "Comic", "Anime"])
generate = st.button("🚀 Generate Comic", use_container_width=True)

# Generation logic
if generate:
    st.subheader("📚 Comic Panels")
    scenes = [s.strip() for s in story.strip().split(".") if s.strip()]
    comic_panels = []

    for i, scene in enumerate(scenes):
        with st.spinner(f"🎬 Generating panel {i+1}..."):
            caption = generate_caption(scene)
            image = generate_image(scene, style_select)
            panel = add_caption_to_image(image, caption)
            comic_panels.append((panel, caption))
            st.image(panel, caption=f"Panel {i+1}: {caption}", use_column_width=True)

            audio_data = generate_voice(caption)
            if audio_data:
                st.audio(audio_data, format='audio/mp3')

    if len(comic_panels) > 0:
        with st.spinner("🧩 Combining panels into comic pages..."):
            image_only_panels = [img for img, _ in comic_panels]
            pages = combine_panels_to_page(image_only_panels)

            for i, page in enumerate(pages):
                page_path = f"comic_page_{i+1}.png"
                page.save(page_path)
                st.subheader(f"🖼️ Comic Page {i+1}")
                st.image(page, use_column_width=True)
                with open(page_path, "rb") as f:
                    st.download_button(f"📥 Download Page {i+1}", f, file_name=page_path, mime="image/png")


Overwriting app.py


In [45]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8506
  Network URL: http://172.28.0.12:8506
  External URL: http://34.16.177.212:8506

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y)   Stopping...
^C


In [28]:
!pip install -q pyngrok


In [29]:
!ngrok config add-authtoken 2zHE52ZCSZCsGLynNigAO07QzOa_4JWg8ETNCvrVteB54mzdR


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [49]:
from pyngrok import ngrok
import threading
import os
import time

# Kill previous tunnels
ngrok.kill()

# Launch Streamlit in background
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

time.sleep(10)  # Wait for server to start

# Setup ngrok
ngrok.set_auth_token("2zHE52ZCSZCsGLynNigAO07QzOa_4JWg8ETNCvrVteB54mzdR")
public_url = ngrok.connect(8501)
print("🔗 Your app is live at:", public_url)


🔗 Your app is live at: NgrokTunnel: "https://d658-34-16-177-212.ngrok-free.app" -> "http://localhost:8501"
